In [1]:
import pandas as pd
# pd.set_option('display.max_rows', None)

import os
from subprocess import call
import numpy as np

In [2]:
import grab_exp_data
EXP_DATE='230821'

In [3]:
def unpivot_df(df, type=None):
        df = df.stack().reset_index()
        df.columns=['experiment', 'component', 'raw conc']
        if not type:
            df['type']=['DNA'] * len(df)
        else: 
            df['type']=['compound'] * len(df)
        return df

In [12]:
file_names = [i for i in os.listdir(EXP_DATE) if "~" not in i]
buffers_mt=[(EXP_DATE+'/'+i) for i in file_names if 'buffers_mt' in i] 
buffersmt_df = pd.read_excel(grab_exp_data.get_most_updated(buffers_mt), index_col=0).dropna(axis=1, how='all')
exp_start_idx = list(buffersmt_df.columns).index('Kan') + 1
exp_list = list(buffersmt_df.columns)[DNA_start_idx:]
buffersmt_df = unpivot_df(buffersmt_df, type=1)
buffersmt_df

,experiment,component,raw conc,type
0,mod1,formate-koh,25.0,compound
1,mod1,THF,1.0,compound
2,mod1,NADPH,1.0,compound
3,mod1,ATP,1.0,compound
4,mod1,Mg-Act,2.0,compound
...,...,...,...,...
71,fdh,NADP,1.0,compound
72,fdh,fdhAT_QH,10.0,compound
73,fdh-neg,formate-koh,25.0,compound
74,fdh-neg,NADP,1.0,compound


In [13]:
exp_list

['THF',
 '5,10-CH2-THF',
 'NADP',
 'NADPH',
 'NADH',
 'ATP',
 'Glycine',
 'AMP',
 'Serine',
 'PLP',
 'Mg-Act',
 'HCT',
 'HCO3',
 'Kan',
 'mod1',
 'glyA',
 'sds',
 'neg',
 'fdhAT_QH']

In [8]:
genex_mt=[(EXP_DATE+'/'+i) for i in file_names if 'genex_mt' in i] 
genemt_df = pd.read_excel(grab_exp_data.get_most_updated(genex_mt), index_col=0).dropna(axis=1, how='all')
DNA_start_idx = list(genemt_df.columns).index('PLP') + 1
DNA_list = list(genemt_df.columns)[DNA_start_idx:]
genemt_df = unpivot_df(genemt_df)
genemt_df

,experiment,component,raw conc,type
0,mod1,PLP,0.50,DNA
1,mod1,fch,1.75,DNA
2,mod1,ftl,3.50,DNA
3,mod1,folD_Aw,8.00,DNA
4,glyA,PLP,0.50,DNA
5,glyA,glyA_Ec,5.00,DNA
6,sds,PLP,0.50,DNA
7,sds,sds,5.00,DNA
8,neg,PLP,0.50,DNA
9,fdhAT_QH,fdhAT_QH,6.00,DNA


In [9]:
DNA_list

['sds', 'glyA_Ec', 'fch', 'ftl', 'folD_Aw', 'fdhAT_QH']

In [10]:
genemt_df['conc(M)'] = np.where(genemt_df['component'].isin(DNA_list), genemt_df['raw conc']*1E-9, genemt_df['raw conc']*1E-3)
genemt_df

,experiment,component,raw conc,type,conc(M)
0,mod1,PLP,0.50,DNA,5.000000e-04
1,mod1,fch,1.75,DNA,1.750000e-09
2,mod1,ftl,3.50,DNA,3.500000e-09
3,mod1,folD_Aw,8.00,DNA,8.000000e-09
4,glyA,PLP,0.50,DNA,5.000000e-04
5,glyA,glyA_Ec,5.00,DNA,5.000000e-09
6,sds,PLP,0.50,DNA,5.000000e-04
7,sds,sds,5.00,DNA,5.000000e-09
8,neg,PLP,0.50,DNA,5.000000e-04
9,fdhAT_QH,fdhAT_QH,6.00,DNA,6.000000e-09


In [11]:
# filter out all the rows that DON'T contain experiment name in 
# component column of bufferdf

ValueError: 'Kan' is not in list

In [25]:
a['conc(M)'] = np.where(a['component'].isin(DNA_list), a['raw conc']*1E-9, a['raw conc']*1E-3)
a

,experiment,component,raw conc,type,conc(M)
0,mod1,formate-koh,25.0,cofactor,2.500000e-02
1,mod1,THF,1.0,cofactor,1.000000e-03
2,mod1,NADPH,1.0,cofactor,1.000000e-03
3,mod1,ATP,1.0,cofactor,1.000000e-03
4,mod1,Mg-Act,2.0,cofactor,2.000000e-03
...,...,...,...,...,...
71,fdh,NADP,1.0,cofactor,1.000000e-03
72,fdh,fdhAT_QH,10.0,cofactor,1.000000e-08
73,fdh-neg,formate-koh,25.0,cofactor,2.500000e-02
74,fdh-neg,NADP,1.0,cofactor,1.000000e-03


In [21]:

clean.import_init_conc_data(EXP_DATE).reset_index()

,experiment,type,component,concentration
0,fdh,cofactor,NADP,1.00
1,fdh,cofactor,fdhAT_QH,10.00
2,fdh,cofactor,formate-koh,25.00
3,fdh-neg,cofactor,NADP,1.00
4,fdh-neg,cofactor,formate-koh,25.00
5,fdh-neg,cofactor,neg,10.00
6,fdhAT_QH,enzyme,fdhAT_QH,6.00
7,glyA,cofactor,"5,10-CH2-THF",1.00
8,glyA,cofactor,Glycine,1.00
9,glyA,cofactor,Kan,1.00


In [5]:
## Extracting peak data
met_data = grab_exp_data.import_final_conc_data(EXP_DATE)
## Converting peak areas into concentrations
grab_exp_data.peak_to_conc(met_data, 'standard_curves/standard_curves_JS.csv')met_data

peak area
experiment metabolite            
fdh        glycine       6798.090
           malate        2651.230
           nadh           589.155
           nadp        847329.725
           nadph       235525.495
...                           ...
sds-neg    nadp          1773.270
           nadph          248.100
           pyruvate     24289.945
           serine      539524.135
           thf            478.330

[110 rows x 1 columns]